<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-26 14:43:58
-------------------
qualified stocks: 93
with latest results: 30
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.34 C
-------------------
Today PnL: 33.45 K (0.25%)
Current PnL: -15.37 L (-10.71%)
CY Booked + Current PnL: -8.88 L (-6.18%)
-------------------
Total profit:  3.63 L
Total loss:  -18.99 L
-------------------
Total Booked + Current PnL: 18.10 L (15.53%)
Total Booked PnL: 33.47 L (28.72%)
Curr Year Booked PnL: 6.49 L (4.85%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 72.25 L (53.94%)
Deployed:  1.17 C
Current:  1.34 C
CAGR/XIRR %: 9.55%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELIANCE,1291.83,1526.00,9.14,X-LC,56.99,162334.0,21525.0,3993.0,1.50,15.29,2.46,18.13,18.0,5.39,1.21,28.18,XY25,NTT,REFINERIES
84,UJJIVANSFB,52.77,53.00,51.69,M-SC,84.95,132084.0,-10395.0,11016.0,-0.91,-7.30,8.34,0.44,242.0,-0.94,0.99,55.90,OX40N,NTT,BANKS
3,ADANIPORTS,1103.69,1583.00,1.05,M-LC,47.31,101317.0,22955.0,11074.0,2.43,29.29,10.93,43.43,79.0,2.07,0.76,36.17,XY24,NTT,MISC
35,HINDALCO,651.95,761.55,-7.59,H-LC,62.37,110392.0,6080.0,11459.0,2.57,5.83,10.38,16.81,27.0,0.53,0.82,22.81,X5K,ATH,METALS
89,VIPIND,488.80,489.00,-950.28,H-SC,95.16,80964.0,-14352.0,14387.0,-1.41,-15.06,17.77,0.04,153.0,-1.00,0.60,59.69,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.87,M-MC,52.15,179669.0,-139.0,112545.0,0.58,-0.08,62.64,62.52,170.0,-0.00,1.34,33.98,XR,NTT,BANKS
50,KPIGREEN,497.21,732.16,NaN,NaN,73.12,125383.0,86.0,59118.0,-0.20,0.07,47.15,47.25,137.0,0.00,0.94,56.86,MH,ATH,POWER
47,JIOFIN,310.58,387.00,-1.28,M-LC,86.02,217945.0,539.0,52961.0,2.65,0.25,24.30,24.61,94.0,0.01,1.63,54.96,XY24,BTT,FINANCE


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,101.81,M-MC,63.44,229921.0,4959.0,162692.0,4.10,2.20,70.76,74.53,186.0,0.03,1.72,33.49,XY24,BTT,STEEL
60,RAJESHEXPO,517.65,518.00,1877.27,M-SC,53.76,53941.0,-83236.0,83328.0,3.34,-60.68,154.48,0.07,232.0,-1.00,0.40,34.15,OX40N,NTT,JEWELLERY
56,NATIONALUM,189.63,247.44,-35.66,H-SC,70.97,102597.0,2093.0,28542.0,3.00,2.08,27.82,30.49,97.0,0.07,0.77,35.16,MH,ATH,METALS
47,JIOFIN,310.58,387.00,-1.28,M-LC,86.02,217945.0,539.0,52961.0,2.65,0.25,24.30,24.61,94.0,0.01,1.63,54.96,XY24,BTT,FINANCE
35,HINDALCO,651.95,761.55,-7.59,H-LC,62.37,110392.0,6080.0,11459.0,2.57,5.83,10.38,16.81,27.0,0.53,0.82,22.81,X5K,ATH,METALS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,2347.98,4546.37,5.23,X-SC,98.92,98023.0,-2940.0,97474.0,-2.42,-2.91,99.44,93.63,65.0,-0.03,0.73,46.00,SR,ATH,CHEMICALS
75,SURYODAY,189.78,240.00,53.51,H-SC,81.72,108917.0,-41009.0,80686.0,-2.35,-27.35,74.08,26.46,143.0,-0.51,0.81,39.53,XR,NTT,BANKS
23,COLPAL,2714.63,3767.14,-3.18,X-MC,16.13,184860.0,-26881.0,108975.0,-2.06,-12.70,58.95,38.77,52.0,-0.25,1.38,1.92,XY25,ATH,FMCG
74,STARHEALTH,551.56,761.00,-6.32,M-SC,5.38,153113.0,-47103.0,123133.0,-1.93,-23.53,80.42,37.97,215.0,-0.38,1.14,22.94,XY24,NTT,INSURANCE
22,COFFEEDAY,59.14,80.00,-51.71,L-SC,95.16,71002.0,-42547.0,82597.0,-1.81,-37.47,116.33,35.27,261.0,-0.52,0.53,73.78,XR,NTT,HOTELS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,53.0,51.69,M-SC,84.95,132084.0,-10395.0,11016.0,-0.91,-7.30,8.34,0.44,242.0,-0.94,0.99,55.90,OX40N,NTT,BANKS
78,TATAELXSI,7332.28,7332.0,-16.32,X-MC,31.18,87794.0,-14858.0,14855.0,0.07,-14.47,16.92,-0.00,58.0,-1.00,0.66,33.46,OX40N,NTT,IT
89,VIPIND,488.80,489.0,-950.28,H-SC,95.16,80964.0,-14352.0,14387.0,-1.41,-15.06,17.77,0.04,153.0,-1.00,0.60,59.69,OX40N,NTT,MISC
42,INDIGOPNTS,1407.73,1408.0,120.42,H-SC,88.17,144572.0,-29987.0,30028.0,-0.78,-17.18,20.77,0.02,128.0,-1.00,1.08,25.37,OX40N,NTT,PAINTS
76,SYMPHONY,1306.42,1306.0,-33.74,M-SC,1.08,141349.0,-29792.0,29740.0,-0.32,-17.41,21.04,-0.03,188.0,-1.00,1.06,1.59,OX40N,NTT,DURABLES


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-17.44,X-LC,29.03,268694.0,-27240.0,121154.0,1.60,-9.20,45.09,31.73,1.0,-0.22,2.01,7.55,X40,BTT,IT
44,INFY,1461.46,2275.00,-9.55,X-LC,59.14,289783.0,28182.0,117449.0,0.25,10.77,40.53,55.67,2.0,0.24,2.16,17.54,X40,BTT,IT
36,HINDUNILVR,2413.81,2723.00,-18.73,X-LC,21.51,255102.0,-15245.0,49872.0,-0.04,-5.64,19.55,12.81,6.0,-0.31,1.91,6.69,XY25,NTT,FMCG
15,BRITANNIA,4983.33,6446.05,16.68,X-LC,67.74,267398.0,38165.0,29120.0,1.84,16.65,10.89,29.35,7.0,1.31,2.00,27.05,XY25,ATH,FMCG
57,NESTLEIND,2268.60,2755.00,-10.69,X-LC,32.26,283468.0,18042.0,38863.0,0.77,6.80,13.71,21.44,9.0,0.46,2.12,13.42,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.0,-28.99,M-SC,87.10,207182.0,35561.0,126505.0,1.92,20.72,61.06,94.43,196.0,0.28,1.55,52.91,XR,NTT,DURABLES
3,ADANIPORTS,1103.69,1583.0,1.05,M-LC,47.31,101317.0,22955.0,11074.0,2.43,29.29,10.93,43.43,79.0,2.07,0.76,36.17,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,101.81,M-MC,63.44,229921.0,4959.0,162692.0,4.10,2.20,70.76,74.53,186.0,0.03,1.72,33.49,XY24,BTT,STEEL
18,CAMS,3643.00,5250.99,5.33,H-SC,76.34,118384.0,16380.0,28649.0,0.09,16.06,24.20,44.14,124.0,0.57,0.88,34.91,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,1.05,M-LC,47.31,101317.0,22955.0,11074.0,2.43,29.29,10.93,43.43,79.0,2.07,0.76,36.17,XY24,NTT,MISC
47,JIOFIN,310.58,387.00,-1.28,M-LC,86.02,217945.0,539.0,52961.0,2.65,0.25,24.30,24.61,94.0,0.01,1.63,54.96,XY24,BTT,FINANCE
92,WIPRO,243.46,420.00,-2.45,M-LC,79.57,167319.0,16374.0,93080.0,0.17,10.85,55.63,72.51,69.0,0.18,1.25,15.26,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-28.99,M-SC,87.10,207182.0,35561.0,126505.0,1.92,20.72,61.06,94.43,196.0,0.28,1.55,52.91,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-2.45,M-LC,79.57,167319.0,16374.0,93080.0,0.17,10.85,55.63,72.51,69.0,0.18,1.25,15.26,XR,NTT,IT
41,INDIAMART,2327.09,4850.92,-46.24,H-SC,93.55,140063.0,16727.0,117037.0,-0.67,13.56,83.56,108.45,121.0,0.14,1.05,38.95,AR,ATH,MISC
56,NATIONALUM,189.63,247.44,-35.66,H-SC,70.97,102597.0,2093.0,28542.0,3.00,2.08,27.82,30.49,97.0,0.07,0.77,35.16,MH,ATH,METALS
50,KPIGREEN,497.21,732.16,NaN,NaN,73.12,125383.0,86.0,59118.0,-0.20,0.07,47.15,47.25,137.0,0.00,0.94,56.86,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-51.71,L-SC,95.16,71002.0,-42547.0,82597.0,-1.81,-37.47,116.33,35.27,261.0,-0.52,0.53,73.78,XR,NTT,HOTELS
54,MASFIN,326.60,402.39,-18.25,H-SC,74.19,92700.0,-5280.0,28014.0,0.00,-5.39,30.22,23.21,141.0,-0.19,0.69,34.45,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,5.23,X-SC,98.92,98023.0,-2940.0,97474.0,-2.42,-2.91,99.44,93.63,65.0,-0.03,0.73,46.00,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,1.05,M-LC,47.31,101317.0,22955.0,11074.0,2.43,29.29,10.93,43.43,79.0,2.07,0.76,36.17,XY24,NTT,MISC
35,HINDALCO,651.95,761.55,-7.59,H-LC,62.37,110392.0,6080.0,11459.0,2.57,5.83,10.38,16.81,27.0,0.53,0.82,22.81,X5K,ATH,METALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7272.22,M-SC,100.00,80860.0,-12950.0,89569.0,-0.46,-13.80,110.77,81.67,240.0,-0.14,0.60,56.59,XR,NTT,CERAMICS
4,ALKYLAMINE,2347.98,4546.37,5.23,X-SC,98.92,98023.0,-2940.0,97474.0,-2.42,-2.91,99.44,93.63,65.0,-0.03,0.73,46.00,SR,ATH,CHEMICALS
48,JPPOWER,18.73,26.20,-33.07,L-SC,97.85,136163.0,-5679.0,62254.0,-0.33,-4.00,45.72,39.88,259.0,-0.09,1.02,37.99,XY24,NTT,POWER
66,SAMMAANCAP,170.35,326.00,-170.94,M-SC,96.77,86178.0,-16032.0,109420.0,0.09,-15.69,126.97,91.37,203.0,-0.15,0.64,39.89,XY25,NTT,FINANCE
22,COFFEEDAY,59.14,80.00,-51.71,L-SC,95.16,71002.0,-42547.0,82597.0,-1.81,-37.47,116.33,35.27,261.0,-0.52,0.53,73.78,XR,NTT,HOTELS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.19
1,25,42.48
2,50,71.13


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.70
LC    37.20
MC    21.52
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.23
XY25     13.41
X40      12.41
X40N     10.60
XR       10.37
OX40N     8.27
AR        7.88
MH        1.71
X5K       1.49
X200      1.45
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.44
X-LC    15.41
M-SC    14.02
H-LC    12.38
M-LC     8.32
H-MC     8.11
X-MC     7.44
M-MC     5.61
X-SC     2.76
L-SC     2.48
L-LC     1.09
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.32,-2.10,36.27
IT,12.01,-10.61,65.54
FINANCE,7.95,-13.18,54.15
BANKS,7.94,-7.30,51.08
MISC,6.67,-21.07,68.29
PAINTS,5.60,-16.36,39.30
ELECTRICAL,5.04,-4.21,49.34
HEALTHCARE,4.51,-6.82,38.03
AUTO,4.40,-14.78,65.26


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2752251.0,25
XR,1070167.0,13
AR,1015564.0,9
X40,658437.0,9
X40N,511505.0,10
XY25,491965.0,9
OX40N,322612.0,11
SR,180244.0,2
MH,87660.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1892797.0,19
M-SC,1451085.0,18
X-LC,602828.0,9
H-LC,597916.0,11
M-MC,592799.0,5
H-MC,435531.0,6
X-MC,421073.0,6
M-LC,331050.0,6
L-SC,265412.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,866570.0,6
M-SC,XY24,599113.0,5
H-SC,AR,517957.0,3
M-MC,XY24,388535.0,3
M-SC,XR,327514.0,4
X-LC,X40,316575.0,3
H-LC,X40N,239889.0,5
H-MC,XY24,232573.0,2
M-LC,XY24,220502.0,4


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
